In [2]:
import math
import pandas as pd
import scipy.stats as st
import numpy as np

In [3]:
213.0968 - 54.64

158.4568

In [4]:
158.4568 / 4

39.6142

In [5]:
54.64 / 3

18.213333333333335

In [6]:
18.213/39.6142

0.45975937921250465

In [14]:
alpha = 0.05
df_1 = 3
df_2 = 28

crit_val = st.f.ppf(1 - alpha, df_1, df_2)
crit_val

2.9466852660172655

In [15]:
val_setup = {
    'groups': [1, 2, 3, 4],
    'count': [8 ,8, 8, 8],
    'average': [5.8, 9, 8, 9]
}

dunn_test = pd.DataFrame(val_setup)
dunn_test.set_index('groups', inplace=True)
dunn_test

,count,average
groups,,
1,8,5.8
2,8,9.0
3,8,8.0
4,8,9.0


In [16]:
k = len(dunn_test) - 1
v = dunn_test['count'].sum() - k
n = 8

print(f"k = {k}\nv = {v}")

dunnett_crit = 2.47  # based on table

# WILL NEED TO ROUND DOWN

k = 3
v = 29


In [17]:
y_bar = dunn_test['average'].mean()

SSW = mse = np.sum((dunn_test['average'] - y_bar)**2)
SSW

6.830000000000002

In [18]:
d = dunnett_crit * math.sqrt((2 * mse) / n)
print(d)

3.22758218330688


In [19]:
dunn_test['average'] - y_bar

groups
1   -2.15
2    1.05
3    0.05
4    1.05
Name: average, dtype: float64

In [40]:
alpha = 0.05
crit_val = st.studentized_range.ppf(1 - alpha, len(dunn_test), v)

print(f"Critical val: {crit_val}")

eqs = dunn_test.T

Critical val: 3.853037659694372


In [53]:
w = crit_val * math.sqrt(mse / n)
w

3.560152587740582

In [61]:
pairs = pd.DataFrame()
calcs = pd.DataFrame()
i = 0
while i < len(eqs.columns):
    j = i + 1
    while j < len(eqs.columns):
        # create pair dictionary to concat with pairs
        pair_dict = {
            "pair": f"{eqs.columns[i]} and {eqs.columns[j]}"
        }
        # create dataframe to append
        pair_df = pd.DataFrame(pair_dict, index=[len(pairs)])
        # concat/append to pairs df
        pairs = pd.concat([pairs, pair_df])
        # print(pairs)
        # concat/append to calcs df
        calcs = pd.concat([calcs, pd.DataFrame(eqs.iloc[:, i] - eqs.iloc[:, j]).sum()], ignore_index=True)
        # print(calcs)
        j += 1    
    i += 1

pairs = pd.concat([pairs, calcs], axis=1)
pairs.sort_values(by=[0], ascending=False, inplace=True)

In [62]:
# pairs[0]

3    1.0
4    0.0
5   -1.0
1   -2.2
0   -3.2
2   -3.2
Name: 0, dtype: float64

In [70]:
pairs["tukey"] = pairs[0].apply(lambda x: "proceed" if x - min(pairs[0]) > w else "stop")

In [71]:
pairs

,pair,0,tukey
3,2 and 3,1.0,proceed
4,2 and 4,0.0,stop
5,3 and 4,-1.0,stop
1,1 and 3,-2.2,stop
0,1 and 2,-3.2,stop
2,1 and 4,-3.2,stop
